In [1]:
import pandas as pd # Beautiful tool for data wrangling! e.g. '!pip install pandas' from a Notebook
# See https://mariadb.com/blog/how-connect-python-programs-mariadb e.g. '!pip install mysql' from Notebook
import MySQLdb 
import re
from collections import Counter
import os
from collections import OrderedDict

pd.set_option("display.max_rows",35) # Useful when having large Pandas DataFrames like we do here

In [2]:
conn = MySQLdb.connect(user='mos', passwd='', db='monuments_db')
cursor = conn.cursor()

In [3]:
cursor.execute("SHOW TABLES")
non_country_tables = set(["prox_search","admin_tree","commonscat","image","monuments_all"])
results = cursor.fetchall()
all_tables = []
for pair in results:
    for item in pair:
        all_tables.append(item)
country_tables =  list(set(all_tables) - non_country_tables) # set difference, then convert back to list
len(country_tables)

103

In [4]:
cnt = Counter()
for table in country_tables:
    sql = 'SELECT count(*) FROM `{}`'.format(table)
    cursor.execute(conn.escape_string(sql))
    res = cursor.fetchone()
    cnt.update({table: res[0]})
empty_tables = []
for table,count in cnt.items():
    if count == 0:
        empty_tables.append(table)
len(empty_tables)

6

In [5]:
final_tables = set(country_tables) - set(empty_tables)
len(final_tables)

97

In [6]:
# test one country specific table  
t = "monuments_in_(en)"
sql = 'SELECT * FROM `{}`'.format(t)
pl = pd.io.sql.read_sql(conn.escape_string(sql), conn)

In [7]:
list(pl.keys())

['number',
 'description',
 'location',
 'address',
 'district',
 'lat',
 'lon',
 'image',
 'commonscat',
 'state_iso',
 'circle',
 'source',
 'changed']

In [31]:
list(pl.ix[0])

['N-AP-1',
 'Hill Fort and buildings therein and the fortifications at the foot of the hill',
 '[[Gooty]]',
 '',
 '[[Anantapur district|Anantapur]]',
 15.119999999999999,
 77.629999999999995,
 'Gooty Fort.JPG',
 '',
 'IN-AP',
 '',
 '//en.wikipedia.org/w/index.php?title=List_of_Monuments_of_National_Importance_in_Andhra_Pradesh&oldid=715348799',
 Timestamp('2016-06-08 06:50:24')]

In [45]:
def create_wikitables(table, df):
    """Takes a Pandas DataFrame object and writes wikitables to files named after the table they are
    produced from to {the current Directory}/wikitables/
    """
    country_fields = list(df.keys())
    country_values = list(df.ix[0]) # get first row in table as example data from 
    country_data = list(zip(country_fields, country_values))
    
    table_header = '{| class="wikitable"\n'
    table_name = '|+ '+table + "\n"
    # create table columns
    table_columns = "! heritage field\n! example value\n! Wikidata property\n! Conversion\n! Comment\n|-\n"
    table_rows = []
    for (field, value) in zip(country_fields, country_values):
        row="| "+ str(field) + "\n| " + str(value) + "\n|\n| \n| \n|-\n"
        table_rows.append(row)
    
    table_rows_str = "".join(table_rows)
    # Fill in examples values from the first record in the table
    table_footer = "\n|}"
    wikitable = table_header + table_name + table_columns + table_rows_str[:-1] + table_footer
    
    if os.path.isdir("./files"):
        out = open("./files/" + table + ".wikitable","w")
        out.write(wikitable)
        out.flush()
        
        print("Directory ./files exists. Wrote file {}".format(out.name))
        out.close()
    else:
        os.mkdir("./files")
        with open("./files/" + table + ".wikitable","w") as out:
            out.write(wikitable)
            print("./files doesn't exist")

In [46]:
for table in final_tables:
    sql = 'SELECT * FROM `{}`'.format(table)
    df = pd.io.sql.read_sql(conn.escape_string(sql), conn)
    create_wikitable(table, df)
#%ls ./files

Directory ./files exists. Wrote file ./files/monuments_ad_(ca).wikitable
Directory ./files exists. Wrote file ./files/monuments_nl-prov_(nl).wikitable
Directory ./files exists. Wrote file ./files/monuments_ve_(es).wikitable
Directory ./files exists. Wrote file ./files/monuments_pa_(es).wikitable
Directory ./files exists. Wrote file ./files/monuments_no_(no).wikitable
Directory ./files exists. Wrote file ./files/monuments_nl-gem_(nl).wikitable
Directory ./files exists. Wrote file ./files/monuments_fr_(ca).wikitable
Directory ./files exists. Wrote file ./files/monuments_lu_(lb).wikitable
Directory ./files exists. Wrote file ./files/se_fornmin.wikitable
Directory ./files exists. Wrote file ./files/monuments_pl-old_(pl).wikitable
Directory ./files exists. Wrote file ./files/monuments_be-bru_(nl).wikitable
Directory ./files exists. Wrote file ./files/monuments_cm_(fr).wikitable
Directory ./files exists. Wrote file ./files/monuments_pk_(en).wikitable
Directory ./files exists. Wrote file ./fi